In [ ]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [ ]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
# os.makedirs(POS_PATH)
# os.makedirs(NEG_PATH)
# os.makedirs(ANC_PATH)

In [ ]:
new_directory = 'C:/Users/LENOVO/FaceID' 

os.chdir(new_directory)

print(os.getcwd())

In [ ]:
# base_path = 'lfw'

# for directory in os.listdir(base_path):
#      for file in os.listdir(base_path, directory):
#          EX_PATH = os.path.join(base_path, directory, file)
#          NEW_PATH = os.path.join(NEG_PATH, file)
#          os.replace(EX_PATH, NEW_PATH)


# base_path = 'lfw'  
# NEG_PATH = 'C:/Users/LENOVO/FaceID/data/negative'  


# for directory in os.listdir(base_path):
#     dir_path = os.path.join(base_path, directory)
    
#     if os.path.isdir(dir_path):
      
#         for file in os.listdir(dir_path):
#             EX_PATH = os.path.join(dir_path, file)  
#             NEW_PATH = os.path.join(NEG_PATH, file)  
            
           
#             os.replace(EX_PATH, NEW_PATH)


In [ ]:

import uuid

In [ ]:
  cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    if cv2.waitKey(10) & 0XFF == ord('a'):
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)

    if cv2.waitKey(10) & 0XFF == ord('p'):
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)

    frame = frame[120:102+250, 200:200+250, :]
    cv2.imshow('Image Collection', frame)
    if cv2.waitKey(10) & 0XFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(300)

In [ ]:
 def preprocess(file_path):
     byte_img = tf.io.read_file(file_path)
     img = tf.io.decode_jpeg(byte_img)
     img = tf.image.resize(img, (100,100))
     img = img/255.0
     return img
     

In [ ]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [ ]:
data

In [ ]:
samples = data.as_numpy_iterator()

In [ ]:
samples.next()

In [ ]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

In [ ]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [ ]:
data

In [ ]:
samples = data.as_numpy_iterator()

In [ ]:
len(samples.next())

In [ ]:
plt.imshow(samples.next()[1])

In [ ]:
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [ ]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [ ]:
def make_enbedding():
    inp = Input(shape=(100,100,3), name = 'input_image')
    c1 = Conv2D(64,(10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64,(2,2), padding='same')(c1)

    c2 = Conv2D(128,(7,7), activation = 'relu')(m1)
    m2 = MaxPooling2D(64,(2,2), padding = 'same')(c2)

    c3 = Conv2D(128,(4,4), activation = 'relu')(m2)
    m3 = MaxPooling2D(64,(2,2), padding = 'same')(c3)

    c4 = Conv2D(256,(4,4), activation = 'relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation = 'sigmoid')(f1)

    return Model(inputs = [inp], outputs = [d1], name = 'embedding')
    

In [ ]:
embedding = make_enbedding()

In [ ]:
embedding.summary()

In [ ]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        input_embedding = tf.convert_to_tensor(input_embedding)
        validation_embedding = tf.convert_to_tensor(validation_embedding)
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
l1 = L1Dist()

In [ ]:
def make_siamese_model():

    input_image = Input(name='input_img', shape=(100,100,3))
    validation_image = Input(name='validation_img', shape=(100,100,3))
    input_embedding = embedding(input_image)
    validation_embedding = embedding(validation_image)
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distance = siamese_layer(input_embedding, validation_embedding)

    classifier = Dense(1, activation = 'sigmoid')(distance)
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='siameseNetwork')

In [ ]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)


In [ ]:
@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        X = batch[:2]
        y = batch[2]

        yhat = siamese_model(X, training = True)
        yhat = tf.squeeze(yhat, axis=0)
        loss = binary_cross_loss(y, yhat)

        grad = tape.gradient(loss, siamese_model.trainable_variables)
        opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        return loss

In [ ]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch{}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))

        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)

        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
    

In [ ]:
EPOCHS = 50

In [ ]:
# train(train_data, EPOCHS)

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
test_var = test_data.as_numpy_iterator().next

In [ ]:
y_hat = siamese_model.predict([test_input, test_val])
y_hat

In [ ]:
[1 if prediction > 0.5 else 0 for prediction in y_hat.flatten()]

In [ ]:
y_true

In [ ]:
m = Recall()
m.update_state(y_true, y_hat)
m.result().numpy()

In [ ]:
siamese_model.save('siamesemodel.keras')

In [ ]:
model = tf.keras.models.load_model('siamesemodel.keras', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
model.summary()

## Real Time Test

In [ ]:

def verify(frame, model, detection_threshold, verification_threshold):
    results = []
    for image in os.listdir(os.path.join('Application_data', 'Verification_image')):
        input_img = preprocess(os.path.join('Application_data', 'Input_image', 'Input_image.jpg'))
        validation_img = preprocess(os.path.join('Application_data', 'Verification_image', image))

        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    detection = np.sum(np.array(results)> detection_threshold)
    verification = detection / len(os.listdir(os.path.join('Application_data', 'verification_image')))
    verified = verification > verification_threshold

    return results, verified

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:102+250, 200:200+250, :]

    cv2.imshow('verified', frame)

    if cv2.waitKey(10) & 0xFF == ord('v'):
        save_path = os.path.join('Application_data', 'input_image', 'input_image.jpg')
        cv2.imwrite(save_path, frame)
        result, verified = verify(frame, model, 0.5, 0.5)
        print(verified)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()